# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.


In [1]:
# 의존성 패키지 설치하기
%pip install pip==24.0.0 -q
%pip install ratsnlp -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 23.3 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.1 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.


In [3]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/content/drive/MyDrive/kdt_240424/m7_nlp 응용/checkpoint-generation1",
)

Exception: downstream_model_dir "/content/drive/MyDrive/kdt_240424/m7_nlp 응용/checkpoint-generation1" is not valid

# 모델 로딩
파인튜닝을 마친 GPT2 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

<ipython-input-4-c3710c30dba0>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fine_tuned_model_ckpt = torch.load(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


- 탑k 샘플링(top-k sampling)은
  - top_k는 확률이 높은 상위 k개의 토큰만을 고려하여 다음 토큰을 선택하는 방법입니다. 즉, 다음 단어의 예측 시 확률이 높은 상위 k개의 토큰 중에서만 선택합니다.
  - 예를 들어, top_k=50이라면 확률이 가장 높은 상위 50개의 토큰만을 고려 대상으로 하고, 이 중에서 다음 단어를 무작위로 선택합니다.
  - top_k는 모델이 너무 예측 가능한 텍스트를 생성하는 것을 방지하고 다양성을 제공하지만, 때로는 관련 없는 단어를 포함시킬 수 있는 단점이 있습니다.

- 탑p 샘플링(top-p sampling)은
  - top_p는 누적 확률이 p 이상이 되는 순간까지 가장 확률이 높은 토큰들을 선택하는 방식입니다. 즉, 선택된 토큰들의 누적 확률이 전체 토큰 확률 분포의 p 퍼센트를 차지할 때까지 토큰을 포함합니다. 이 방법을 핵 샘플링(Nucleus Sampling)이라고도 합니다.
  - 예를 들어, top_p=0.9라면 확률이 높은 순서대로 토큰을 추가하다가 누적 확률이 90%에 도달하는 순간 멈춥니다. 이렇게 선택된 토큰들 중에서 무작위로 다음 토큰을 선택합니다.
  - top_p는 생성된 텍스트의 다양성을 증가시킬 수 있으면서도, 너무 무작위적이지 않게 제어할 수 있는 장점이 있습니다.
- 리피티션 패널티(repetition penalty)라는 방식으로 반복을 통제할 수도 있습니다. repetition_penalty라는 인자를 주면 됩니다. 그 값은 1.0 이상이어야 하며 클 수록 페널티가 세게 적용

- 템퍼러처 스케일링(temperature scaling)이란
  - 모델의 다음 토큰 확률분포에 변형을 가해 문장을 다양하게 생성하는 기법
  - 확률분포를 변형한다는 의미는, 대소 관계의 역전 없이 분포의 모양만을 바꾼다는 의미
  - 이 값이 0에 가까울 수록 확률분포 모양이 원래 대비 뾰족해 진다. 순위의 변동은 없지만 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아져 확률분포의 모양이 뾰족(sharp)해진다. 그만큼 확률값 기준 1등 토큰이 다음 토큰으로 뽑힐 가능성이 높아진다. temperature의 기술적 범위는 0을 제외한 양수 전체
  - temperature를 1보다 작게 하면 상대적으로 정확한 문장을, 1보다 크게 하면 상대적으로 다양한 문장을 생성한다.A higher temperature will result in more random predictions, while a lower temperature will result in more confident predictions.

- no_repeat_ngram_size 매개변수는
  - 언어 모델이 텍스트를 생성할 때 특정 크기의 n-gram이 반복되지 않도록 하는 기능을 설정합니다. n-gram은 인접한 n개의 아이템(이 경우 단어)으로 구성된 시퀀스입니다. 이 매개변수는 생성된 텍스트 내에서의 반복을 줄이고, 다양성과 창의성을 높이는 데 도움을 줄 수 있습니다.
  - no_repeat_ngram_size가 0이면: 이 기능이 비활성화됩니다. 즉, 모델이 텍스트를 생성할 때 어떤 크기의 n-gram도 반복될 수 있습니다.
  - no_repeat_ngram_size가 2 이상의 정수로 설정된 경우: 지정된 크기의 n-gram이 생성된 텍스트 내에서 한 번만 나타나도록 합니다. 예를 들어, no_repeat_ngram_size를 2로 설정하면 어떤 두 단어의 조합도 텍스트 내에서 단 한 번만 나타날 수 있습니다. 이는 특히 장기적인 텍스트 생성에서 반복되는 패턴이나 구문의 반복을 방지하는 데 유용합니다.


# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0, # 확률적으로 선택되는 후보군의 누적 확률의 임계값. top_p가 1.0인 경우, 모든 후보가 선택
        top_k=50, # 각 단계에서 고려할 확률이 높은 상위 k개의 토큰
        repetition_penalty=1.0, # 반복되는 단어에 대한 패널티를 조정합니다. 1.0보다 크면 반복을 억제
        no_repeat_ngram_size=0, # 생성된 텍스트 내에서 반복되지 않아야 하는 n-gram의 크기
        temperature=1.0, # 생성 다양성을 조절. 값이 낮을수록 예측이 보수적
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True, # 토큰 생성 시 샘플링을 통해 다양성을 높입니다.
                top_p=float(top_p), # 생성 과정에서 이 임계값을 넘는 토큰만을 고려
                top_k=int(top_k), # 각 단계에서 고려할 상위 k개의 토큰
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty), # 반복되는 토큰에 대한 패널티를 적용
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature), # 값이 낮을수록 예측이 더 확정적이 됩니다.
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br>
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [ ]:
inference_fn(
        prompt='안녕하세요',
        min_length=10,
        max_length=50,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        no_repeat_ngram_size=3,
        temperature=1.0,
)

{'result': '안녕하세요.....이영화 재밌었다</s>'}

In [ ]:
inference_fn(
        prompt='안녕하세요',
        min_length=10,
        max_length=50,
        top_p=0.5,
        top_k=10,
        repetition_penalty=1.0,
        no_repeat_ngram_size=2,
        temperature=0.1,
)

{'result': '안녕하세요..이거보고나서..정말재밌었어요</s>'}